### Health Check

In [13]:
$Info             = Get-ComputerInfo 
$InfoProp         = $Info | Select-Object -Property OsLastBootUpTime,BiosReleaseDate,TimeZone,WindowsInstallDateFromRegistry,
             OsName,OsOperatingSystemSKU,OSArchitecture,WindowsVersion,WindowsBuildLabEx,OsMuiLanguages,BiosCaption,CsBootupState,CsDNSHostName,CsDomain,CsProcessors,OsSystemDirectory

$additionalDetails= [ordered]@{                                                               #Calculating additional system details
    'TimeZone'    = (Get-TimeZone).Id
    'System Time' = Get-Date
    'PhysicalHost'= (Get-Item "HKLM:\SOFTWARE\Microsoft\Virtual Machine\Guest\Parameters").GetValue("PhysicalHostName")
    'NW Adapters' = {$Info.CsNetworkAdapters.Description -join ', '}.Invoke()
    'IPAddress'   = {$Info.CsNetworkAdapters.IPAddresses -join ', '}.Invoke()

    'Cpu Usage %' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'Mem Usage %' = { [Math]::Ceiling(($Info.OsTotalVisibleMemorySize - $Info.OsFreePhysicalMemory) * 100 / $Info.OsTotalVisibleMemorySize) }.Invoke()
    'C: % Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
}

$InfoList         = $InfoProp.PSObject.Properties | ForEach-Object { @{ $_.Name = $_.Value } }#Converting properties to key-value pairs
$combinedData     = $InfoList + $additionalDetails                                            # Combining both sets of data into one list
$combinedData | Format-Table -HideTableHeaders -AutoSize                                      # Displaying the combined list


OsLastBootUpTime               12/4/2023 9:24:25 AM
BiosReleaseDate                7/11/2023 7:00:00 PM
TimeZone                       (UTC-06:00) Central Time (US & Canada)
WindowsInstallDateFromRegistry 3/7/2023 12:48:05 AM
OsName                         Microsoft Windows 11 Enterprise
OsOperatingSystemSKU           EnterpriseEdition
OsArchitecture                 ARM 64-bit Processor
WindowsVersion                 2009
WindowsBuildLabEx              22621.1.arm64fre.ni_release.220506-1250
OsMuiLanguages                 {en-US}
BiosCaption                    Hyper-V UEFI Release v4.1
CsBootupState                  Normal boot
CsDNSHostName                  W11ARM
CsDomain                       WORKGROUP
CsProcessors                   {Ampere(R) Altra(R) Processor}
OsSystemDirectory              C:\Windows\system32
TimeZone                       Central Standard Time
System Time                    12/4/2023 6:54:00 PM
PhysicalHost                   CHI231053017064
NW Adapters        

This shows which processes are consuming the most cpu, memory, network and disk and have the most number of handles and threads currently on the system.

In [10]:
#$Host.UI.RawUI.BufferSize = New-Object Management.Automation.Host.Size(127, $Host.UI.RawUI.BufferSize.Height)  #v2: One line equivalent

$objArr         = Get-Process|Select @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads

$exclude        = { $_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle" }
$Network        = ((Get-Counter '\Process(*)\IO Data Bytes/sec').CounterSamples.Where($exclude)|Sort cookedvalue -Descending -Top 10).instancename

$combinedIO     = @{}                                                     # Get IO Read and Write Operations
$ioOps          = (Get-Counter '\Process(*)\IO Read Operations/sec', '\Process(*)\IO Write Operations/sec').CounterSamples.Where($exclude)
foreach ($op in $ioOps) {$combinedIO[$op.InstanceName] += $op.CookedValue}# Combine and sum the IO operations for each process
$Disk           = ($combinedIO.GetEnumerator() | Sort Value -Descending -Top 10).Name

$sortCriteria   = @('CPU', 'WS', 'HandleCount', 'Threads')                # Using a loop to sort for each property
$sortedResults  = @{}
foreach ($criteria in $sortCriteria) {$sortedResults[$criteria] = $objArr | Sort-Object $criteria -Descending -Top 10}

$harray         = for ($j = 0; $j -lt 10; $j++) {                         # Creating the custom object array
    [PSCustomObject][ordered]@{
        'CPU'   = $sortedResults['CPU'][$j].Process 
        'WS'    = $sortedResults['WS'][$j].Process 
        'Handle'= $sortedResults['HandleCount'][$j].Process 
        'Thread'= $sortedResults['Threads'][$j].Process
        'Net'   = $Network[$j]
        'Disk'  = $Disk[$j]
    }
}
$harray | Format-Table -AutoSize


CPU                    WS                      Handle                 Thread                  Net               Disk
---                    --                      ------                 ------                  ---               ----
msedge►9832            Memory Compression►2744 System►4               System►4                code              code
msedge►8864            dwm►2276                explorer►5572          Memory Compression►2744 code              msedge
Code►13792             msedge►8864             OneDrive►11396         explorer►5572           registry          dotnet
Code►4200              Code►4200               msedge►8864            SearchHost►7748         code              dwm
msedge►8040            msedge►8040             lsass►900              msedge►8864             code              svchost
OneDrive►11396         Grammarly.Desktop►5228  Grammarly.Desktop►5228 Code►12224              code              healthservice
Grammarly.Desktop►5228 pwsh►3740               M